# SPU的简单使用

## 1.初始化

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'carol', 'dave'], address='local')
aby3_config = sf.utils.testing.cluster_def(parties=['alice', 'bob', 'carol'])
spu_device = sf.SPU(aby3_config)
alice, bob = sf.PYU('alice'), sf.PYU('bob')

The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-28 20:08:23,123	INFO worker.py:1724 -- Started a local Ray instance.


(_run pid=685158) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=685158) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=685158) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


## 2.某一个参与方执行指定函数

执行结果为明文形式

In [2]:
def debit_amount(x, y):
    return x + y


debit_amount_pyu = alice(debit_amount)(1,2)
print(debit_amount_pyu)
print(sf.reveal(debit_amount_pyu))

3


## 3.某一参与方读取私有数据

### 3.1 使用内置读取方法

仅支持读取二进制序列化数据（如用pickle保存的数据）

In [3]:
import pickle

# 创建一个示例字典
data = {
    'name': 'Alice',
    'age': 30,
    'city': 'New York'
}

# 将对象序列化并保存到文件
with open('/home/beng003/python_project/sf-test/data/seri_alice.txt', 'wb') as f:
    pickle.dump(data, f)

print("数据已序列化并保存到文件。")


数据已序列化并保存到文件。


In [4]:
seri_alice=alice.load('/home/beng003/python_project/sf-test/data/seri_alice.txt')
print(seri_alice)
print(sf.reveal(seri_alice))

{'name': 'Alice', 'age': 30, 'city': 'New York'}


### 3.2 搭配常用的数据读取函数读取数据 

利用原理为让指定参与方执行指定函数

In [5]:
import pandas as pd

m_v_alice = pd.read_csv('/home/beng003/python_project/sf-test/data/v_alice.csv')
print(m_v_alice.head())
print(type(m_v_alice))

   age           job  marital  education default  balance housing loan    uid
0   49  entrepreneur  married  secondary      no     1160     yes   no   2831
1   30   blue-collar  married    primary      no     1480     yes   no   3102
2   30    technician   single  secondary      no     1286      no   no  23752
3   31        admin.  married  secondary      no       23      no   no  14479
4   33    technician  married   tertiary      no     -258      no   no  13052
<class 'pandas.core.frame.DataFrame'>


In [6]:
v_alice=alice(pd.read_csv)('/home/beng003/python_project/sf-test/data/v_alice.csv')
print(v_alice)
print(sf.reveal(v_alice).head())

   age           job  marital  education default  balance housing loan    uid
0   49  entrepreneur  married  secondary      no     1160     yes   no   2831
1   30   blue-collar  married    primary      no     1480     yes   no   3102
2   30    technician   single  secondary      no     1286      no   no  23752
3   31        admin.  married  secondary      no       23      no   no  14479
4   33    technician  married   tertiary      no     -258      no   no  13052


### 3.3 直接将指定数据传给指定参与方

In [7]:
import jax.numpy as jnp


alice_data = jnp.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
alice_data_pyu = sf.to(alice, alice_data)

print(alice_data_pyu)
print(sf.reveal(alice_data_pyu))

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


[0 1 2 3 4 5 6 7 8 9]


## 4. 将数据进行多方共享

共享的数据类型必须是数值类型

In [8]:
alice_data_spu=alice_data_pyu.to(spu_device)

print(alice_data_spu)
print(sf.reveal(alice_data_spu))

[0 1 2 3 4 5 6 7 8 9]


## 5. 执行多方计算

Value xxxxxx with dtype <U3 is not a valid JAX array type. Only arrays of numeric types are supported by JAX."

In [9]:
def apply_operator(a, b):
    return a+b, a-b, a*b, a/b


bob_data = jnp.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
bob_data_pyu = sf.to(bob, bob_data)
bob_data_spu = bob_data_pyu.to(spu_device)

选项 1：将所有返回值视为单一返回值
这是 SPU 的默认行为
选项 2：即时决定返回值数量
我们还可以指示 SPU 为我们决定返回值数量。
选项 3：手动确定返回值数量
如果可能，您还可以手动设置返回值数量。

```python
def __call__(
    self,
    func: Callable,
    *,
    static_argnames: Union[str, Iterable[str], None] = None,
    num_returns_policy: SPUCompilerNumReturnsPolicy = SPUCompilerNumReturnsPolicy.SINGLE,
    user_specified_num_returns: int = 1,
    copts: spu_pb2.CompilerOptions = spu_pb2.CompilerOptions(),
):
```

In [10]:
from secretflow.device.device.spu import SPUCompilerNumReturnsPolicy
alice_add_bob_data, alice_sub_bob_data, alice_mul_bob_data, alice_div_bob_data= (
    spu_device(apply_operator, num_returns_policy=SPUCompilerNumReturnsPolicy.FROM_COMPILER)(alice_data_spu, bob_data_spu)
)

print(alice_add_bob_data)
print(sf.reveal(alice_add_bob_data))
print(alice_sub_bob_data)
print(sf.reveal(alice_sub_bob_data))
print(alice_mul_bob_data)
print(sf.reveal(alice_mul_bob_data))
print(alice_div_bob_data)
print(sf.reveal(alice_div_bob_data))

[ 0  2  4  6  8 10 12 14 16 18]
[0 0 0 0 0 0 0 0 0 0]
[ 0  1  4  9 16 25 36 49 64 81]
[0.         0.99999666 0.9999966  0.99998647 0.9999966  0.9999974
 0.99998647 0.9999999  0.9999966  1.        ]
